# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
#azdias = pd.read_csv('Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv', sep=';')

In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!
customers.head()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

### Data Exploration:
    1. Observe datatypes
    2. Find percentage of NaN values

In [ ]:
customers.dtypes.unique()

In [ ]:
null_count = []
for i in customers:
    value = customers[i].isnull().sum()
    null_count.append(value)

print("Percentage of null values in each column:")
for i in range(len(null_count)-1):
    print("{}: {:.2f}%".format(customers.columns[i], (null_count[i]/customers.shape[0] * 100)))

In [ ]:
#for i in customers:
#    print(i)
#    print(customers[i].unique())

### Data Processing

In [ ]:
import statistics
import random
def preprocessing (dataframe):
    dataframe = dataframe.copy()
    dataframe.drop_duplicates(keep = 'first', inplace = True)
    dataframe.replace(-1, float('NaN'), inplace=True)
    dataframe.replace(0, float('NaN'), inplace=True)
    
    
    new_list = []

    for i in range(0, dataframe.shape[1]):     
        if (dataframe.iloc[:, i].dtypes == 'object'):
            dataframe.iloc[:, i] = pd.Categorical(dataframe.iloc[:, i])
            dataframe.iloc[:, i] = dataframe.iloc[:, i].cat.codes 
            dataframe.iloc[:, i] = dataframe.iloc[:, i].astype('object')

            new_list.append(dataframe.columns[i])
            
    for i in dataframe:
        dataframe[i] = dataframe[i].fillna(dataframe[i].median())
    

    customers.drop(['LNR'], axis=1, inplace=True)
    
    return dataframe

In [ ]:
cleaned_customers = preprocessing(customers)
cleaned_customers.head()

In [ ]:
# Dropping specific columns with greater than 60% NaN values
cleaned_customers.drop(['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4'], axis=1, inplace=True)

### Data Processing Confirmation

In [ ]:
null_count = []
for i in cleaned_customers:
    value = cleaned_customers[i].isnull().sum()
    null_count.append(value)

print("Percentage of null values in each column:")
for i in range(len(null_count)-1):
    print("{}: {:.2f}%".format(cleaned_customers.columns[i], (null_count[i]/cleaned_customers.shape[0] * 100)))

In [ ]:
cleaned_population = preprocessing(azdias)
cleaned_population.head()

In [ ]:
#    The preferred method for imputation is KNN Imputation.
#    However, due to the large dataset, it is computationally expensive. 
#    The choice was made not to include KNN imputation, and opt for 
#    imputation using median values. 
#    
#    Below is the code that would've been used if the dataset weren't too
#    large. 

#from sklearn.impute import KNNImputer # Importing K Nearest Neighbors Algorithm
# K Nearest Neighbours algorithm is used to replace values with its nearest neighbours - or most similar row data
#df_final = pd.DataFrame(KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean').fit(cleaned_customers).transform(cleaned_customers), columns = cleaned_customers.columns)

In [ ]:
#for i in cleaned_customers:
#    print(cleaned_customers[i].unique())

# PCA 

### Customers PCA and Kmeans clustering

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
X_df = pca.fit(cleaned_customers).transform(cleaned_customers)
pca.explained_variance_ratio_

In [ ]:
from sklearn.preprocessing import StandardScaler

PC = range(1, pca.n_components_+1)
plt.bar(PC, pca.explained_variance_ratio_, color='gold')
plt.xlabel('Principal Components')
plt.ylabel('Variance %')
plt.xticks(PC)

# Putting components in a dataframe for later
PCA_components = pd.DataFrame(X_df)

In [ ]:
pca = PCA(n_components=2)
pca_customers = pca.fit(cleaned_customers).transform(cleaned_customers)
pca.explained_variance_ratio_

In [ ]:
from sklearn.cluster import KMeans # Importing K-Means algorithm 
from sklearn.impute import KNNImputer # Importing K Nearest Neighbors Algorithm
from sklearn.linear_model import LinearRegression # Importing Linear Regression model 
from sklearn.metrics import mean_squared_error # Evaluation metric 
from sklearn.model_selection import train_test_split # Preprocessing for training and testing data splits 

from yellowbrick.cluster import KElbowVisualizer # Importing Elbow Method Library

model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,15)) # Loop through model to find ideal number of clusters within the data

visualizer.fit(PCA_components)
visualizer.show()
%time

In [ ]:
from sklearn.cluster import KMeans
model2 = KMeans(n_clusters = 3, init = "k-means++")
y2 = model2.fit_predict(X_df)
plt.figure(figsize=(15,15))
label2 = model2.fit_predict(X_df)
uniq = np.unique(label2)
for i in uniq:
  plt.scatter(X_df[label2 == i , 0] , X_df[label2 == i , 1] , label = i)
plt.xlabel([])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.show()

In [ ]:
cleaned_customers['cluster'] = model2.labels_ # Adding extra column to the location dataframe to allocate data to separate groups
cleaned_customers.head()

In [ ]:
array = model2.cluster_centers_
array = array.astype(int)
array

In [ ]:
customers['CAMEO_DEUG_2015'].unique()

AGER_TYP	best-ager typology
ALTERSKATEGORIE_GROB - age classification through prename analysis 
ANREDE_KZ - gender
BALLRAUM - distance to next urban centre 
CAMEO_DEUG_2015 - class
D19_BANKEN_ANZ_12 - transaction activity BANKS in the last 12 months
D19_BANKEN_ANZ_24 -transaction activity BANKS in the last 24 months
D19_BANKEN_DATUM - actuality of the last transaction for the segment banks TOTAL
     
D19_VERSAND_OFFLINE_DATUM	actuality of the last transaction for the segment mail-order OFFLINE
D19_VERSAND_ONLINE_DATUM	actuality of the last transaction for the segment mail-order ONLINE
D19_VERSAND_DATUM	actuality of the last transaction for the segment mail-order TOTAL
D19_VERSAND_ONLINE_QUOTE_12	amount of online transactions within all transactions in the segment mail-order 
D19_VOLLSORTIMENT_RZ	transactional activity based on the product group COMPLETE MAIL-ORDER OFFERS
D19_BANKEN_ONLINE_DATUM
D19_BANKEN_ONLINE_QUOTE_12
FINANZ_MINIMALIST	financial typology: low financial interest
FINANZ_SPARER	financial typology: money saver
FINANZ_VORSORGER	financial typology: be prepared
FINANZ_ANLEGER	financial typology: investor
FINANZ_UNAUFFAELLIGER	financial typology: unremarkable
FINANZ_HAUSBAUER	financial typology: main focus is the own house
FINANZTYP	best descirbing financial type for the peron
GEBURTSJAHR	year of birth
GFK_URLAUBERTYP	vacation habits
GREEN_AVANTGARDE	the environmental sustainability is the dominating movement in the youth of these consumers
HEALTH_TYP	health typology
LP_LEBENSPHASE_FEIN	lifestage fine
LP_LEBENSPHASE_GROB	lifestage rough
LP_FAMILIE_FEIN	family type fine
LP_FAMILIE_GROB	family type rough
LP_STATUS_FEIN	social status fine 
LP_STATUS_GROB	social status rough
NATIONALITAET_KZ	nationaltity
PRAEGENDE_JUGENDJAHRE	dominating movement in the person's youth (avantgarde or mainstream)
RETOURTYP_BK_S	return type
SEMIO_SOZ	affinity indicating in what way the person is social minded
SEMIO_FAM	affinity indicating in what way the person is familiar minded
SEMIO_REL	affinity indicating in what way the person is religious
SEMIO_MAT	affinity indicating in what way the person is material minded
SEMIO_VERT	affinity indicating in what way the person is dreamily
SEMIO_LUST	affinity indicating in what way the person is sensual minded
SEMIO_ERL	affinity indicating in what way the person is eventful orientated
SEMIO_KULT	affinity indicating in what way the person is cultural minded
SEMIO_RAT	affinity indicating in what way the person is of a rational mind
SEMIO_KRIT	affinity indicating in what way the person is critical minded
SEMIO_DOM	affinity indicating in what way the person is dominant minded
SEMIO_KAEM	affinity indicating in what way the person is of a fightfull attitude
SEMIO_PFLICHT	affinity indicating in what way the person is dutyfull traditional minded
SEMIO_TRADV	affinity indicating in what way the person is traditional minded
SHOPPER_TYP	shopping typology
SOHO_FLAG	small office/home office flag
TITEL_KZ	flag whether this person holds an academic title
VERS_TYP	insurance typology 
ZABEOTYP	typification of energy consumers
GEBAEUDETYP_RASTER	industrial areas
KKK	purchasing power
MOBI_REGIO	moving patterns
ONLINE_AFFINITAET	online affinity
REGIOTYP	AZ neighbourhood typology

In [ ]:
dataframe_cluster_0 = cleaned_customers[cleaned_customers['cluster'] == 0]
dataframe_cluster_1 = cleaned_customers[cleaned_customers['cluster'] == 1]
dataframe_cluster_2 = cleaned_customers[cleaned_customers['cluster'] == 2]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#define data
data = [len(dataframe_cluster_0), len(dataframe_cluster_1), len(dataframe_cluster_2)]
labels = ['Cluster 1', 'Cluster 2', 'Cluster 3']

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:5]

#create pie chart
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc_df = cleaned_customers.copy()
rfc_y = rfc_df.pop('cluster')
rfc_X = rfc_df[:]
rfc_X.head()

In [ ]:
rfc_y.head()

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
rfc_X_train, rfc_X_test, rfc_y_train, rfc_y_test = train_test_split(rfc_X, rfc_y)
from sklearn.svm import SVC
rfc = RandomForestClassifier()
rfc.fit(rfc_X_train, rfc_y_train)
rfc_pred = rfc.predict(rfc_X_test)
# Add pipeline for grid search to optimize model
from sklearn.metrics import accuracy_score
print ("Accuracy : {:.2f}%".format(accuracy_score(rfc_y_test, rfc_pred)*100))

In [ ]:
rfc_array = rfc.feature_importances_

In [ ]:
#df = pd.DataFrame(array.reshape(1, 368), columns=X.columns)
rfc_importances = []
count = 0
for i in rfc_array:
    rfc_importances.append([i, rfc_X.columns[count]])
    count += 1

In [ ]:
rfc_importances.sort(reverse=True)
rfc_labels = []
rfc_values = []
for i in rfc_importances[0:20]:
    rfc_labels.append(i[1])
    rfc_values.append(i[0])
rfc_importances

In [ ]:
sns.barplot(rfc_labels, rfc_values)
plt.xticks(rotation=90)

# Exploratory Data Analysis

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Distribution of Gender in Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['ANREDE_KZ'],  ax=axes[0], color ='red', bins = 2, kde = False)

sns.distplot(dataframe_cluster_1['ANREDE_KZ'],  ax=axes[1], color ='red', bins = 2, kde = False)

sns.distplot(dataframe_cluster_2['ANREDE_KZ'],  ax=axes[2], color ='red', bins = 2, kde = False)

### Observations:
- The majority of customers are male. 
- The first cluster has a more even distribution of males to females as compared to the other two clusters. 

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Distribution of Age in Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['ALTERSKATEGORIE_GROB'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['ALTERSKATEGORIE_GROB'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['ALTERSKATEGORIE_GROB'],  ax=axes[2], color ='red', bins = 10, kde = False)

### Oberservations:
- The first cluster has a younger population with individuals falling into the category of less than 30 years of age, and between the ages of 30 and 45. 
- The second cluster tends to have a larger population of middle aged individuals ranging from 46 - 60 years of age. 
- The third cluster has the largest percentage of individuals who are over the age of 60 years of age

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Transaction level of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['D19_VERSAND_ONLINE_DATUM'],  ax=axes[0], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_1['D19_VERSAND_ONLINE_DATUM'],  ax=axes[1], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_2['D19_VERSAND_ONLINE_DATUM'],  ax=axes[2], color ='red', bins = 20, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Distance from City Center of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['BALLRAUM'],  ax=axes[0], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_1['BALLRAUM'],  ax=axes[1], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_2['BALLRAUM'],  ax=axes[2], color ='red', bins = 20, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Online Transaction Activity of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['D19_VERSAND_ONLINE_DATUM'],  ax=axes[0], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_1['D19_VERSAND_ONLINE_DATUM'],  ax=axes[1], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_2['D19_VERSAND_ONLINE_DATUM'],  ax=axes[2], color ='red', bins = 20, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Online Transactions within all Transactions in the Segment Mail-Order of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['D19_VERSAND_ONLINE_QUOTE_12'],  ax=axes[0], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_1['D19_VERSAND_ONLINE_QUOTE_12'],  ax=axes[1], color ='red', bins = 20, kde = False)

sns.distplot(dataframe_cluster_2['D19_VERSAND_ONLINE_QUOTE_12'],  ax=axes[2], color ='red', bins = 20, kde = False)

#sns.distplot(dataframe_cluster_3['D19_VERSAND_ONLINE_QUOTE_12'],  ax=axes[1, 1], color ='red', bins = 20, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Rough Social Status of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['LP_STATUS_GROB'],  ax=axes[0], color ='red', bins = 3, kde = False)

sns.distplot(dataframe_cluster_1['LP_STATUS_GROB'],  ax=axes[1], color ='red', bins = 3, kde = False)

sns.distplot(dataframe_cluster_2['LP_STATUS_GROB'],  ax=axes[2], color ='red', bins = 3, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Good Social Status of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['LP_STATUS_FEIN'],  ax=axes[0], color ='red', bins = 3, kde = False)

sns.distplot(dataframe_cluster_1['LP_STATUS_FEIN'],  ax=axes[1], color ='red', bins = 3, kde = False)

sns.distplot(dataframe_cluster_2['LP_STATUS_FEIN'],  ax=axes[2], color ='red', bins = 3, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Class of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['CAMEO_DEUG_2015'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['CAMEO_DEUG_2015'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['CAMEO_DEUG_2015'],  ax=axes[2], color ='red', bins = 10, kde = False)


In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Number of Cars in Postal Code Area of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['KBA13_ANZAHL_PKW'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['KBA13_ANZAHL_PKW'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['KBA13_ANZAHL_PKW'],  ax=axes[2], color ='red', bins = 10, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Number of Households in Postal Code Area of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['PLZ8_HHZ'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['PLZ8_HHZ'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['PLZ8_HHZ'],  ax=axes[2], color ='red', bins = 10, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Development of Most Recent Car Segment in Postal Code Area of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['KBA05_MODTEMP'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['KBA05_MODTEMP'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['KBA05_MODTEMP'],  ax=axes[2], color ='red', bins = 10, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15,5))
figure.suptitle('Development of Most Recent Car Manufacturer in Postal Code Area of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['KBA05_HERSTTEMP'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['KBA05_HERSTTEMP'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['KBA05_HERSTTEMP'],  ax=axes[2], color ='red', bins = 10, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Number of Buildings in Postal Code Area of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['PLZ8_GBZ'],  ax=axes[0], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_1['PLZ8_GBZ'],  ax=axes[1], color ='red', bins = 10, kde = False)

sns.distplot(dataframe_cluster_2['PLZ8_GBZ'],  ax=axes[2], color ='red', bins = 10, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Share of Cars Per Household of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['KBA13_AUTOQUOTE'],  ax=axes[0], color ='red', bins = 5, kde = False)

sns.distplot(dataframe_cluster_1['KBA13_AUTOQUOTE'],  ax=axes[1], color ='red', bins = 5, kde = False)

sns.distplot(dataframe_cluster_2['KBA13_AUTOQUOTE'],  ax=axes[2], color ='red', bins = 5, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Residential Area of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['WOHNLAGE'],  ax=axes[0], color ='red', bins = 8, kde = False)

sns.distplot(dataframe_cluster_1['WOHNLAGE'],  ax=axes[1], color ='red', bins = 8, kde = False)

sns.distplot(dataframe_cluster_2['WOHNLAGE'],  ax=axes[2], color ='red', bins = 8, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Unemployment of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['RELAT_AB'],  ax=axes[0], color ='red', bins = 6, kde = False)

sns.distplot(dataframe_cluster_1['RELAT_AB'],  ax=axes[1], color ='red', bins = 6, kde = False)

sns.distplot(dataframe_cluster_2['RELAT_AB'],  ax=axes[2], color ='red', bins = 6, kde = False)

In [ ]:
figure, axes = plt.subplots(1, 3, sharex=True, figsize=(15, 5))
figure.suptitle('Unemployment of Each Cluster', fontsize=20)

sns.distplot(dataframe_cluster_0['KBA13_KMH_0_140'],  ax=axes[0], color ='red', bins = 5, kde = False)

sns.distplot(dataframe_cluster_1['KBA13_KMH_0_140'],  ax=axes[1], color ='red', bins = 5, kde = False)

sns.distplot(dataframe_cluster_2['KBA13_KMH_0_140'],  ax=axes[2], color ='red', bins = 5, kde = False)

In [ ]:
filtered_customers = cleaned_customers.filter(['AGER_TYP', 'ALTERSKATEGORIE_GROB', 'ANREDE_KZ', 'BALLRAUM', 'D19_BANKEN_ANZ_12', 'D19_BANKEN_ANZ_24', 
                                       'D19_BANKEN_ANZ_24',
                          'D19_BANKEN_DATUM', 'D19_VERSAND_OFFLINE_DATUM', 'D19_VERSAND_ONLINE_DATUM', 'D19_VERSAND_DATUM',
                           'D19_VERSAND_ONLINE_QUOTE_12', 'FINANZ_MINIMALIST', 'FINANZ_SPARER', 'FINANZ_VORSORGER',
                          'FINANZ_ANLEGER', 'FINANZ_UNAUFFAELLIGER', 'FINANZ_HAUSBAUER', 'FINANZTYP', 'GEBURTSJAHR',
                          'GFK_URLAUBERTYP', 'GREEN_AVANTGARDE', 'HEALTH_TYP', 'LP_LEBENSPHASE_FEIN',
                          'LP_LEBENSPHASE_GROB', 'LP_FAMILIE_FEIN', 'LP_FAMILIE_GROB', 'LP_STATUS_FEIN',
                          'LP_STATUS_GROB', 'NATIONALITAET_KZ', 'PRAEGENDE_JUGENDJAHRE', 'RETOURTYP_BK_S',
                          'SEMIO_SOZ', 'SEMIO_FAM', 'SEMIO_REL', 'SEMIO_MAT', 'SEMIO_VERT', 'SEMIO_LUST',
                           'SEMIO_ERL', 'SEMIO_KULT', 'SEMIO_RAT', 'SEMIO_KRIT', 'SEMIO_DOM', 'SEMIO_KAEM',
                          'SEMIO_PFLICHT', 'SEMIO_TRADV', 'SHOPPER_TYP', 'SOHO_FLAG', 'TITEL_KZ', 
                           'VERS_TYP', 'ZABEOTYP', 'GEBAEUDETYP_RASTER', 'KKK', 'MOBI_REGIO',
                           'ONLINE_AFFINITAET', 'REGIOTYP'])

for i in filtered_customers:
        filtered_customers[i] = filtered_customers[i].astype(int)

In [ ]:
#sns.set_theme(style="white")

# Obtaining correlation matrix
#corr_df = filtered_customers.copy() #.drop(['cluster'], axis=1)
corr = filtered_customers.corr()

# Matplotlib graph setup 
f, ax = plt.subplots(figsize=(20, 20))

# Generating Seaplot colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 1}, fmt=".2f")

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
train_csv = pd.read_csv('Udacity_MAILOUT_052018_TRAIN.csv', sep=';')
train_csv.head()

In [ ]:
y = train_csv.pop('RESPONSE')
X = train_csv[:]
X = preprocessing(X)
X.head()

In [ ]:
print(y.count())
print(y.nunique())
print(y.head())

def response_counter(response_array):
    number_of_yes = 0 
    number_of_no = 0
    for i in response_array:
        if i == 1:
            number_of_yes += 1
        else:
            number_of_no += 1
    return number_of_yes, number_of_no

number_of_yes, number_of_no = response_counter(y)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
X_df = pca.fit(X).transform(X)
pca.explained_variance_ratio_

In [ ]:
from sklearn.preprocessing import StandardScaler
#std_wine = StandardScaler().fit_transform(filtered_customers)

#pca = PCA(n_components=20)
#principalComponents = pca.fit_transform(filtered_customers)

PC = range(1, pca.n_components_+1)
plt.bar(PC, pca.explained_variance_ratio_, color='gold')
plt.xlabel('Principal Components')
plt.ylabel('Variance %')
plt.xticks(PC)

# Putting components in a dataframe for later
PCA_components = pd.DataFrame(X_df)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_df = pca.fit(X).transform(X)
pca.explained_variance_ratio_

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

In [ ]:
response_counter(y_test)

In [ ]:
response_counter(y_train)

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto', kernel='sigmoid')
clf.fit(X_train, y_train)

In [ ]:
# Add pipeline for grid search to optimize model
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print ("Accuracy : {:.2f}%".format(accuracy_score(y_test, y_pred)*100))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average=None)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, average=None)

In [ ]:
response_counter(y_pred)

## Predicting Purchasing Patterns on Customers

In [ ]:
predicted_dataframe_cluster_1 = dataframe_cluster_0.copy()
predicted_dataframe_cluster_2 = dataframe_cluster_1.copy()
predicted_dataframe_cluster_3 = dataframe_cluster_2.copy()

In [ ]:
def prediction_function(dataframe):
    dataframe.drop(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP', 'cluster'], axis=1, inplace=True)
    pca = PCA(n_components=2)
    X_component = pca.fit(dataframe).transform(dataframe)
    y_pred = clf.predict(X_component)
    dataframe['prediction'] = y_pred
    number_of_yes, number_of_no = response_counter(dataframe['prediction'])
    print("Percentage of Responses: {:.2f}%".format(number_of_yes/(number_of_yes + number_of_no)))

In [ ]:
prediction_function(predicted_dataframe_cluster_1)

In [ ]:
prediction_function(predicted_dataframe_cluster_2)

In [ ]:
prediction_function(predicted_dataframe_cluster_3)

## Test Dataset

In [ ]:
test_csv = pd.read_csv('Udacity_MAILOUT_052018_TEST.csv', sep=';')
test_csv.head()

In [ ]:
X = test_csv[:]

In [ ]:
y_pred = clf.predict(X)

In [ ]:
test_csv['cluster'] = model2.labels_

In [ ]:
test_csv.head()

### Total Consumer Base

In [ ]:
cleaned_azdias = preprocessing(azdias)
cleaned_azdias.head()

In [ ]:
azdias_df = pca.fit(cleaned_azdias).transform(cleaned_azdias)
pca.explained_variance_ratio_

In [ ]:
label2 = model2.fit_predict(azdias_df)

In [ ]:
cleaned_azdias['cluster'] = model2.labels_
cleaned_azdias.head()

In [ ]:
cleaned_azdias['prediction'] = y_pred
number_of_yes, number_of_no = response_counter(cleaned_azdias['prediction'])
print("Percentage of Responses: {:.2f}%".format(number_of_yes/(number_of_yes + number_of_no)))

# Conclusion: 

From the data we can conclude that the first cluster has the highest chance of purchasing the mail-order product. These tend to be younger individuals, from area codes where the number of cars is smaller and the number of buildings in the postal codes are fewer than the two other groups of individuals. 

This may hint at the idea that the most likely purchasers of the mail-order product are younger individuals below the ages of 46 and that are living in areas experiencing economic growth as the number of new car manufacturers and buildings are increasing. The data also suggests that this group of people make up the majority of its customers - which is a positive sign. 